In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

################################# N O T E S  ###################################
#                         scarpe con tacco 5 pages                             #  
#                          scarpe basse 4 pages                                #
#                           sandali bassi 2 pages                              #
#                           All the others 1 page                              #
#   Shoes having two or more colors have an additional listed page per color   #
################################################################################

In [2]:
# Dictionary with
# key = tail of URL
# value = number of webpages

CATEGORIES = {"sneakers": 1, 
              "sandali-bassi": 2, 
              "stivali-stivaletti": 1, 
              "zeppe": 1, 
              "sandali-con-tacco": 1, 
              "scarpe-con-tacco": 5, 
              "scarpe-basse": 4}


df_kammi = pd.DataFrame(columns = ["id", "nome_prodotto", "categoria", "prezzo", "tacco", "taglie", "colore", "link"])


In [3]:
# Cycle through each category
for category in CATEGORIES.keys():
    numero = 1
    print(CATEGORIES[category])

    # Cycle through each page of the category
    for pages in range(CATEGORIES[category]):
        page = "/pag{numero}/"
        url = f"https://www.kammi.it/scarpe-donna/{category}{page}"
        numero += 1
        response = requests.get(url)
        bs = BeautifulSoup(response.content, 'html.parser')
        container = bs.find_all("a", href = True)
        l_shoe_links = []
        
        # Take shoe link and put into l_shoe_links
        for a in container:
            if a.has_attr("onclick"):
                l_shoe_links.append(a["href"])

        # Cycle through each shoe's page and take the infos
        for shoe_link in l_shoe_links: 
            response = requests.get(shoe_link)
            bs = BeautifulSoup(response.content, 'html.parser')
            id = shoe_link.split("-")[-1].replace(".html", "")
            product_name = bs.find("h1").text
            temp = bs.find_all("p", class_ = "mt-4")
            heel = temp[1].text.strip().split(": ")[1].lower().split(" cm")[0]
            price = bs.find("p", class_ = "price mt-2").text
            sizes = []
            temp = bs.find_all("div", class_ = "option option-taglia")
            color = ""

            for t in temp:
                sizes.append(t.find("button").text)

            sizes = " ".join(sizes)

            color = bs.find("div", class_ = "option-color selected")
            
            # Take the color if possible, else "-"
            if color:
                for c in color:
                    if c.find("div") != -1:
                        color = c.find("div")["style"].split(":")[1].replace(";", "").strip()
            else:
                color = "-"

            # New row for the dataframe
            row = pd.Series({"id": id, 
                    "nome_prodotto": product_name, 
                    "categoria": category,
                    "prezzo": price, 
                    "tacco": heel, 
                    "taglie": sizes, 
                    "colore": color,
                    "link": shoe_link})
            
            df = pd.DataFrame(row).T
            
            # Append row to dataframe
            df_kammi = pd.concat([df_kammi, df])



1
2
1
1
1
5
4


In [4]:
df_kammi =df_kammi.set_index("id")

In [5]:
df_kammi.to_csv("kammi.csv")

In [11]:
df_kammi = pd.read_csv("kammi_pulito.csv")

In [12]:
df_kammi

,id,nome_prodotto,categoria,prezzo,tacco,taglie,colore,link
0,idp21594,Zeppa 3326,Zeppe,95.00 €,4,35 36 37 38 39 40 41,Marrone,https://www.kammi.it/scarpe-donna/zeppe/zeppa-...
1,idp20615,Zeppa L3402,Zeppe,74.90 €,6,40 41,Oro,https://www.kammi.it/scarpe-donna/zeppe/zeppa-...
2,idp20566,Zeppa 100,Zeppe,49.90 €,7,35 36 38,Rosso,https://www.kammi.it/scarpe-donna/zeppe/zeppa-...
3,idp21643,Zeppa 340,Zeppe,69.90 €,5,35 36 37 38 39 40,-,https://www.kammi.it/scarpe-donna/zeppe/zeppa-...
4,idp21608,Zeppa 902,Zeppe,69.90 €,5,36 37 38 39 40,-,https://www.kammi.it/scarpe-donna/zeppe/zeppa-...
...,...,...,...,...,...,...,...,...
204,idp21626,Ciabatta 271,Sandali Bassi,79.90 €,0,35 36 37 38 39 40,Rosa,https://www.kammi.it/scarpe-donna/sandali-bass...
205,idp21610,Ciabatta 407,Sandali Bassi,45.00 €,1,37 38 39 40 41,Verde,https://www.kammi.it/scarpe-donna/sandali-bass...
206,idp21623,Ciabatta new D,Sandali Bassi,25.00 €,0,36 37 38 39 40 41,Verde,https://www.kammi.it/scarpe-donna/sandali-bass...
207,idp21623,Ciabatta new D,Sandali Bassi,25.00 €,0,36 37 38 39 40 41,Verde,https://www.kammi.it/scarpe-donna/sandali-bass...
